# Optical Flow

+ Optical flow is the pattern of apparent motion of image objects between two consecutive frames caused by the movement of object or camera
+ Optical flow analysis has a few assumptions:
 + The pixel intensities of an object do not change between consecutive frames
 + Neighbouring pixels have similar motion
+ The optical flow methods in opencv will first take in a given set of points and a frame.
+ Then it will attempt to find those points in the next frame
+ It is up to the user to supply the points to track
+ We can use Gunner Farneback's algorithm (also built in to OpenCV) to calculate dense optical flow.
+ This dense optical flow will calculate flow for all points in an image.
+ It will color them black in no flow (no movement) is detected.
+ The Lucas-Kanade computes optical flow for a sparse feature set.
 + Meaning only the points it was told to track.

In [1]:
import cv2
import numpy as np

## Lucas Kanade Optical Flow

In [3]:
corner_track_params = dict(maxCorners=10, qualityLevel=0.3, minDistance=7,
                          blockSize=7)

In [4]:
corner_track_params

{'maxCorners': 10, 'qualityLevel': 0.3, 'minDistance': 7, 'blockSize': 7}

In [5]:
# Lucas-Kanade params
lk_params = dict(winSize=(200,200), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [9]:
capture = cv2.VideoCapture(0)


ret,prev_frame = capture.read()

prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

# PTS To Track
prevPts = cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)

mask = np.zeros_like(prev_frame)


while True:
    
    ret,frame = capture.read()
    
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts,None,**lk_params)
    
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        
        x_new, y_new = np.uint8(new.ravel())
        x_prev, y_prev = np.uint8(prev.ravel())
        
        mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev), (0,255,0),3)
        frmae = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
        
    img = cv2.add(frame,mask)
    cv2.imshow("Tracking",img)
    
    k = cv2.waitKey(30) & 0xFF
    if k==27:
        break
        
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)
    

cv2.destroyAllWindows()
capture.release()
    

## Densce Optical Flow

In [12]:
capture = cv2.VideoCapture(0)


ret,frame1 = capture.read()

prvsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[::,1] = 255


while True:
    
    ret,frame2 = capture.read()
    
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg,None,0.5,3,15,3,5,1.2,0)
    
    mag, ang = cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees=True)
    
    hsv_mask[:,:,0] = ang/2
    
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow("Frame",bgr)
    
    k = cv2.waitKey(10) & 0xFF
    if k == 27:
        break
    
    prvsImg = nextImg
    
cv2.destroyAllWindows()
capture.release()